# EmoSonics RateSounds-GUI

This GUI uses the sound models (abstract, vocal) developed for the AudioMostly paper
They have been exported into a py file...
On import, this should start superCollider and create an SC class to interact with.

The GUI itself does...

In [1]:
%pylab
import ipywidgets as widgets
from functools import partial
from IPython.display import display
import pandas
import time, socket

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
import imp
sonmod = imp.load_source('sonmod', 'EmoSonics-soundmodels.py')
# I moved it from .py to here to accomodate the non notebook version of rating
print "wait for SC to start..."
time.sleep(5); # wait until SC is started...
print "resume: establish OSC interfaces..."

wait for SC to start...
resume: establish OSC interfaces...


In [3]:
sm = 'abstract'
ipAddr = socket.gethostbyname(socket.getfqdn())
#sonmod.playevent(sm, [0.8,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5])

In [4]:
# study related service functions
def savedata(dataset, prefix="user-soundmodel-run-"):
    df = pandas.DataFrame(data, 
        columns=['uid', 'snd', 'run', 'step', 'time', 'parvec', 'emotion', 'intensity'])
    df.to_csv(prefix + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)
    data.append([userid.value, soundmodel.value, run.value, 
                 count, time.time()-start_time, v, 
                 wRBtarget.value, wRBintensity.value])
    



def mutate(parent, sigma=0.1):
    d=size(parent)
    print parent
    print "size is " + str(d)
    child = clip((parent + sigma*numpy.random.randn(d)), 0, 1)
    return child

def create_next_generation(parentvec, sigma, nr_of_children=4):
    global generation_counter, v
    generation_counter += 1
    return [parentvec] + [mutate(parentvec, sigma) for k in range(nr_of_children)]

## 6. EmoSonics Sound Rating Interface
### 6.1. Admin-Interface

In [5]:
log_sigma_initval = -0.8
log_sigma_step    = -0.2

#targetlist = ['happy', 'surprised', 'angry', 'disgusted', 'sad', 'calm'];
targetlist = ['happy', 'surprised', 'angry', 'afraid', 'neutral', 'disgusted', 
              'sad', 'tired', 'calm'];
degreelist = ['-', 'weak', 'average', 'strong']


studyprefix= widgets.Text( value="rating-emo9", description='studyprefix:', padding=10); 
userid     = widgets.IntText( value=3001, description='UserID:', padding=10); 
username   = widgets.Text( value="myname", description='Username:', padding=10); 
soundmodel = widgets.ToggleButtons(value="vocal", padding=10,
                description='Sound model:', options=['abstract', 'vocal'])
run        = widgets.IntText(value=0, description='Run (>0):', padding=10);
nrStim     = widgets.IntText(value=10, description='NrStim:', padding=10);

display(studyprefix, userid, username, soundmodel, run, nrStim)

### 6.2. User-Interface

In [6]:
# initialize run
data = []  # time, target state, generation_counter, sigma, parvec, ratings
count = 0
start_time = time.time()

v = sonmod.parvecunmap(sonmod.parspecs[soundmodel.value], 
                   sonmod.parspecs[soundmodel.value]['default'].tolist())

def reset_settings():
    """GUI resets / updates on clicking next"""
    global wRBtarget, wRBintensity, count, start_time;
    if(soundmodel.value=='vocal'):    ps = sonmod.parspec_vocal
    if(soundmodel.value=='abstract'): ps = sonmod.parspec_abstract
    wRBtarget.value = 'neutral'
    wRBintensity.value = '-'
    wT.value= "%d / %d" % (count, nrStim.value)
    
def playfn(val): 
    global v
    sonmod.playevent(soundmodel.value, v)

wBplay = widgets.Button(description='play', value="4", padding=10, width=150)
wBplay.on_click(playfn)

wT = widgets.HTML(value="step: - / -")

wRBtarget = widgets.ToggleButtons(
    description='emotion',
    options=targetlist,
    padding=10)

wRBintensity = widgets.ToggleButtons(
    description='degree',
    options=['-', 'weak', 'average', 'strong'],
    padding=10)

def submit(arg):
    global v, data, count
    data.append([userid.value, soundmodel.value, run.value, 
                 count, time.time()-start_time, v, 
                 wRBtarget.value, wRBintensity.value])
    count +=1;
    reset_settings()
    v = mutate(v, 0.4)
    playfn(0)
    if(count>nrStim.value-1):
        print "completed. thanks."
        fname = "../data/%s-%s-run%d" % (username.value, soundmodel.value, run.value)
        savedata(data, fname)
        print "data saved to " + fname
        print "Please inform the operator."

wBnext = widgets.Button(description='next', padding=10)
wBnext.on_click(submit)

display(wT, wBplay)
display(wRBtarget, wRBintensity, wBnext)
reset_settings()

# EmoSonic User Interface - Please rate the sounds

In [8]:
import OSC,threading

In [9]:
pf = 'rating-emo9'
uid = '3001'
un = 'myname'
run = 0
ns = '4'

In [16]:

data = []
class Rating_receiver:
    def __init__(self, ip = '192.168.0.3', port = 7100, num_emotion = 9, num_degree = 4):
        self.receive_address = ip, port
        self.num_emotion = num_emotion
        self.num_degree = num_degree
        self.choice = targetlist[0]
        self.degree = degreelist[0]
        self.count = 0
        
     
    def spawn(self):
        print"Server Created."
        self.receiveServer = OSC.OSCServer(self.receive_address) # create a serve to receive OSC from the tablet
        self.receiveServer.addDefaultHandlers()
        
    def initilisation_handler(self, addr, tags, stuff, source):
        global sm, pf, uid, un, run, ns
        pf = stuff[0]
        uid = stuff[1]
        un = stuff[2]
        sm = stuff[3]
        run = int(stuff[4])
        ns = stuff[5]
        print stuff

    # Emotion (Happy, surprised,..) change 
    def emotion_handler(self, addr, tags, stuff, source):
        self.choice = targetlist[stuff[0]]
        print stuff[0]
        print self.choice
    # Degree (Weak,average,..) change 
    def degree_handler(self, addr, tags, stuff, source):
        self.degree = degreelist[stuff[0]]
        print self.degree

    # Play the sound based on the parameters
    def play_handler(self, addr, tags, stuff, source):
        global v, sm
        print "inside play event "
        v = mutate(v, 0.4)
        playfn(0)
        #sonmod.playevent(sm, v)
        

    # Save the sound parameter, reset the settings and move to next sound rating
    def next_handler(self, addr, tags, stuff, source):        
        global v, data
        data.append([uid, sm, run, 
                     self.count, time.time()-start_time, v, 
                     self.choice, self.degree])
        reset_settings()
        v = mutate(v, 0.4)
        playfn(0)
        self.count += 1
        print "next next handler"

    # Save the sound in a csv file 
    def save_handler(self, addr, tags, stuff, source):
        global data, un, run
        print "completed. thanks."
        fname = "../data/%s-%s-run%d" % (un, sm, run) # All 3 needs to be replaced. 
#         savedata(data, fname)
        df = pandas.DataFrame(data, 
            columns=['uid', 'snd', 'run', 'step', 'time', 'parvec', 'emotion', 'intensity'])
        df.to_csv(fname + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)
        data.append([userid.value, sm, run, 
                     self.count, time.time()-start_time, v, 
                     self.choice, self.degree])
        print "data saved to " + fname
        print "Please inform the operator."
        self.count = 0 # Reset counter. 

    
    def add_handler(self):
        self.receiveServer.addMsgHandler("/play", self.play_handler)
        self.receiveServer.addMsgHandler("/next", self.next_handler)
        self.receiveServer.addMsgHandler("/emo", self.emotion_handler)
        self.receiveServer.addMsgHandler("/degree", self.degree_handler)
        self.receiveServer.addMsgHandler("/save", self.save_handler)
        self.receiveServer.addMsgHandler("/init", self.initilisation_handler)
            
    def print_registered_func(self):
        for addr in self.receiveServer.getOSCAddressSpace():
            print addr
            
    def start(self):
        # Start OSCServer
        print "\nStarting OSCServer."
        self.emorating_oscServer = threading.Thread(target = self.receiveServer.serve_forever)
        self.emorating_oscServer.start()
        print "\nOSCServer established."
        
    def stop(self):
        # Close the OSC server
        print "\nClosing OSCServer."
        self.receiveServer.close()
        print "Waiting for Server-thread to finish"
        try:
            self.emorating_oscServer.join() ##!!!
            print "Done"
        except AttributeError:
            print "Done"


In [18]:
init_receiver = Rating_receiver (ip = ipAddr, port = 8022,num_emotion = 9, num_degree = 4)
init_receiver.spawn()
init_receiver.add_handler()
init_receiver.start()

Server Created.

Starting OSCServer.

OSCServer established.
inside play event 
[ 0.21511348  0.1128178   0.49460968  0.27672138  1.          0.46390929
  0.43955879  0.02493704  0.20854812  0.03447285  0.62726647  0.87329051]
size is 12
inside play event 
[ 0.0482954   0.          0.36488765  0.2824059   0.68535281  1.
  0.50085106  0.63622876  0.          0.40324368  0.64126115  0.57319236]
size is 12
inside play event 
[ 0.31178994  0.          0.03113507  0.13548067  0.87124897  1.
  0.48563593  0.63732474  0.23870848  0.69457725  0.50455044  1.        ]
size is 12
inside play event 
[ 0.4584182   0.10988999  0.          0.55534942  0.89306523  1.          0.
  1.          0.88462538  1.          0.59727488  1.        ]
size is 12
inside play event 
[ 0.10163944  0.32630468  0.56082522  0.06332985  1.          0.49859412
  0.30344119  0.88455034  0.77873419  1.          0.64689622  1.        ]
size is 12
-
inside play event 
[ 0.4588041   0.81987377  1.          0.0026706   0.84034

In [12]:
print ipAddr
print un
print run
print ns
print sm


129.70.149.124
myname
0
4
abstract
['rating-emo9', '3001', 'myname', 'abstract', '2', '4']
0
happy
1
surprised
[ 0.76826678  0.          0.63333333  0.          0.          0.46153846
  0.5         0.          0.          0.625       0.5         0.37499999]
size is 12


OSCServer: IndexError on request from ip-129-70-147-185.wlan.dyn.cit-ec.net:48631: index 11 is out of bounds for axis 0 with size 11


In [19]:
try:
    init_receiver.stop()
except:
    print "OK"


Closing OSCServer.
Waiting for Server-thread to finish
Done


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/sozen/anaconda/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/sozen/anaconda/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/sozen/anaconda/lib/python2.7/site-packages/OSC.py", line 1816, in serve_forever
    self.handle_request()	# this times-out when no data arrives.
  File "/Users/sozen/anaconda/lib/python2.7/SocketServer.py", line 271, in handle_request
    fd_sets = _eintr_retry(select.select, [self], [], [], timeout)
  File "/Users/sozen/anaconda/lib/python2.7/SocketServer.py", line 150, in _eintr_retry
    return func(*args)
error: (9, 'Bad file descriptor')



In [ ]:
rating_receiver = Rating_receiver(ip = ipAddr, port = 7110, num_emotion = 9, num_degree = 4)
rating_receiver.spawn() # create a receiver server. 
rating_receiver.add_handler()

In [ ]:
# initialize run
data = []  # time, target state, generation_counter, sigma, parvec, ratings
count = 0
start_time = time.time()

v = sonmod.parvecunmap(sonmod.parspecs[soundmodel.value], 
                   sonmod.parspecs[soundmodel.value]['default'].tolist())
rating_receiver.start()

In [ ]:
rating_receiver.stop()
